In [1]:
from pyspark.sql import SparkSession

In [2]:
spark=SparkSession.builder.appName("CustomerDataProcessing").master("yarn").getOrCreate()

25/11/08 20:24:23 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
df=spark.read.format("csv").option("header","true").load("/data/customers-300.csv")

In [4]:
df.show()

+-----------+-----------+---------+-----------+-------+-----------------+---------+
|customer_id|       name|     city|      state|country|registration_date|is_active|
+-----------+-----------+---------+-----------+-------+-----------------+---------+
|          0| Customer_0|     Pune|Maharashtra|  India|       2023-01-19|     True|
|          1| Customer_1|     Pune|West Bengal|  India|       2023-08-10|     True|
|          2| Customer_2|    Delhi|Maharashtra|  India|       2023-08-05|     True|
|          3| Customer_3|   Mumbai|  Telangana|  India|       2023-06-04|     True|
|          4| Customer_4|    Delhi|  Karnataka|  India|       2023-03-15|    False|
|          5| Customer_5|  Kolkata|West Bengal|  India|       2023-08-19|     True|
|          6| Customer_6|  Kolkata| Tamil Nadu|  India|       2023-04-21|    False|
|          7| Customer_7|   Mumbai|  Telangana|  India|       2023-05-23|     True|
|          8| Customer_8|     Pune| Tamil Nadu|  India|       2023-07-17|   

In [5]:
df.printSchema()
from pyspark.sql.functions import *

root
 |-- customer_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- country: string (nullable = true)
 |-- registration_date: string (nullable = true)
 |-- is_active: string (nullable = true)



In [6]:
df=df.withColumn('registration_date',to_date(df['registration_date'],'yyyy-MM-dd'))\
.withColumn('is_active',df['is_active'].cast('boolean'))

In [7]:
df.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- country: string (nullable = true)
 |-- registration_date: date (nullable = true)
 |-- is_active: boolean (nullable = true)



In [8]:
df.fillna('unknown',subset=['city','state','country']).show()

+-----------+-----------+---------+-----------+-------+-----------------+---------+
|customer_id|       name|     city|      state|country|registration_date|is_active|
+-----------+-----------+---------+-----------+-------+-----------------+---------+
|          0| Customer_0|     Pune|Maharashtra|  India|       2023-01-19|     true|
|          1| Customer_1|     Pune|West Bengal|  India|       2023-08-10|     true|
|          2| Customer_2|    Delhi|Maharashtra|  India|       2023-08-05|     true|
|          3| Customer_3|   Mumbai|  Telangana|  India|       2023-06-04|     true|
|          4| Customer_4|    Delhi|  Karnataka|  India|       2023-03-15|    false|
|          5| Customer_5|  Kolkata|West Bengal|  India|       2023-08-19|     true|
|          6| Customer_6|  Kolkata| Tamil Nadu|  India|       2023-04-21|    false|
|          7| Customer_7|   Mumbai|  Telangana|  India|       2023-05-23|     true|
|          8| Customer_8|     Pune| Tamil Nadu|  India|       2023-07-17|   

In [9]:
df=df.withColumn('year',year(df['registration_date']))\
.withColumn('month',month(df['registration_date']))

In [10]:
df.show()

+-----------+-----------+---------+-----------+-------+-----------------+---------+----+-----+
|customer_id|       name|     city|      state|country|registration_date|is_active|year|month|
+-----------+-----------+---------+-----------+-------+-----------------+---------+----+-----+
|          0| Customer_0|     Pune|Maharashtra|  India|       2023-01-19|     true|2023|    1|
|          1| Customer_1|     Pune|West Bengal|  India|       2023-08-10|     true|2023|    8|
|          2| Customer_2|    Delhi|Maharashtra|  India|       2023-08-05|     true|2023|    8|
|          3| Customer_3|   Mumbai|  Telangana|  India|       2023-06-04|     true|2023|    6|
|          4| Customer_4|    Delhi|  Karnataka|  India|       2023-03-15|    false|2023|    3|
|          5| Customer_5|  Kolkata|West Bengal|  India|       2023-08-19|     true|2023|    8|
|          6| Customer_6|  Kolkata| Tamil Nadu|  India|       2023-04-21|    false|2023|    4|
|          7| Customer_7|   Mumbai|  Telangana|  I

In [11]:
distinct_cities=df.select(count_distinct(df['city'])).show()

+--------------------+
|count(DISTINCT city)|
+--------------------+
|                   8|
+--------------------+



In [12]:
distinct_countries=df.select(count_distinct(df['country'])).show()

+-----------------------+
|count(DISTINCT country)|
+-----------------------+
|                      1|
+-----------------------+



In [13]:
distinct_states=df.select(count_distinct(df['state'])).show()

+---------------------+
|count(DISTINCT state)|
+---------------------+
|                    7|
+---------------------+



In [14]:
cust_per_city=df.groupBy('city').count()
cust_per_city.show()

+---------+------+
|     city| count|
+---------+------+
|    Delhi|661025|
|  Kolkata|660174|
|Hyderabad|662281|
|Bangalore|661013|
|Ahmedabad|660218|
|  Chennai|660249|
|   Mumbai|661241|
|     Pune|660737|
+---------+------+



In [15]:
cust_per_state=df.groupBy('state').count()
cust_per_state.show()

+-----------+------+
|      state| count|
+-----------+------+
|    Gujarat|755197|
|      Delhi|755715|
|  Karnataka|754548|
|  Telangana|755534|
|Maharashtra|755937|
| Tamil Nadu|754563|
|West Bengal|755444|
+-----------+------+



In [16]:
cust_per_country=df.groupBy('country').count()
cust_per_country.show()

+-------+-------+
|country|  count|
+-------+-------+
|  India|5286938|
+-------+-------+



In [17]:
df.groupBy(['state','is_active']).count().show()

+-----------+---------+------+
|      state|is_active| count|
+-----------+---------+------+
| Tamil Nadu|     true|376923|
|  Karnataka|     true|377657|
|      Delhi|     true|378450|
|Maharashtra|     true|377654|
|Maharashtra|    false|378283|
|  Telangana|     true|377790|
|  Telangana|    false|377744|
|    Gujarat|     true|377433|
| Tamil Nadu|    false|377640|
|  Karnataka|    false|376891|
|    Gujarat|    false|377764|
|      Delhi|    false|377265|
|West Bengal|    false|377153|
|West Bengal|     true|378291|
+-----------+---------+------+



In [18]:
df.groupBy('state').pivot('is_active',['true','false']).count().show()

+-----------+------+------+
|      state|  true| false|
+-----------+------+------+
|    Gujarat|377433|377764|
|      Delhi|378450|377265|
|  Karnataka|377657|376891|
|  Telangana|377790|377744|
|Maharashtra|377654|378283|
| Tamil Nadu|376923|377640|
|West Bengal|378291|377153|
+-----------+------+------+



In [19]:
df.groupBy('city').pivot('is_active',['true','false']).count().show()

+---------+------+------+
|     city|  true| false|
+---------+------+------+
|    Delhi|330416|330609|
|  Kolkata|329955|330219|
|Hyderabad|331963|330318|
|Bangalore|330873|330140|
|Ahmedabad|329944|330274|
|  Chennai|330368|329881|
|   Mumbai|330079|331162|
|     Pune|330600|330137|
+---------+------+------+



In [20]:
df.filter(df['registration_date']>'2023-02-02').show()

+-----------+-----------+---------+-----------+-------+-----------------+---------+----+-----+
|customer_id|       name|     city|      state|country|registration_date|is_active|year|month|
+-----------+-----------+---------+-----------+-------+-----------------+---------+----+-----+
|          1| Customer_1|     Pune|West Bengal|  India|       2023-08-10|     true|2023|    8|
|          2| Customer_2|    Delhi|Maharashtra|  India|       2023-08-05|     true|2023|    8|
|          3| Customer_3|   Mumbai|  Telangana|  India|       2023-06-04|     true|2023|    6|
|          4| Customer_4|    Delhi|  Karnataka|  India|       2023-03-15|    false|2023|    3|
|          5| Customer_5|  Kolkata|West Bengal|  India|       2023-08-19|     true|2023|    8|
|          6| Customer_6|  Kolkata| Tamil Nadu|  India|       2023-04-21|    false|2023|    4|
|          7| Customer_7|   Mumbai|  Telangana|  India|       2023-05-23|     true|2023|    5|
|          8| Customer_8|     Pune| Tamil Nadu|  I

In [21]:
df.groupBy('city').agg(min(df['registration_date']).alias('oldest customer'),max(df['registration_date']).alias('newest customer')).show()

+---------+---------------+---------------+
|     city|oldest customer|newest customer|
+---------+---------------+---------------+
|    Delhi|     2023-01-01|     2023-12-31|
|  Kolkata|     2023-01-01|     2023-12-31|
|Hyderabad|     2023-01-01|     2023-12-31|
|Bangalore|     2023-01-01|     2023-12-31|
|Ahmedabad|     2023-01-01|     2023-12-31|
|  Chennai|     2023-01-01|     2023-12-31|
|   Mumbai|     2023-01-01|     2023-12-31|
|     Pune|     2023-01-01|     2023-12-31|
+---------+---------------+---------------+



In [22]:
df.write.mode('overwrite').parquet('/data/customersProcessed.csv')

In [23]:
orders=spark.read.format("csv").option("header","true").option("inferSchema","false").load('/data/orders.csv')

In [24]:
orders.show()

+--------+-----------+----------+------------------+---------+
|order_id|customer_id|order_date|      total_amount|   status|
+--------+-----------+----------+------------------+---------+
|       0|    4943169|2024-05-16| 772.6478285123148|Cancelled|
|       1|    3028005|2024-05-03| 986.4670000758517|  Pending|
|       2|    3048147|2024-02-22| 963.2485257551176|  Pending|
|       3|    3388736|2024-01-13| 927.2796436009864|  Shipped|
|       4|    2365317|2024-10-06| 771.1260998047123|  Pending|
|       5|     184420|2024-05-20| 946.3405156142144|  Pending|
|       6|     930983|2024-01-03|335.08604596526186|  Shipped|
|       7|    3268112|2024-06-24| 519.0223162007812|  Shipped|
|       8|    4477369|2024-06-27| 718.8000089550775|Cancelled|
|       9|    3518522|2024-03-13|194.54401365636681|  Pending|
|      10|    1526008|2024-11-17| 272.9056216636697|  Shipped|
|      11|    3591167|2024-11-15| 846.6333789988046|Delivered|
|      12|    2772236|2024-04-25|234.38148667624944|Del

In [25]:
customer_orders=df.join(orders,'customer_id')

In [26]:
customer_orders.show()

+-----------+----------------+---------+-----------+-------+-----------------+---------+----+-----+--------+----------+------------------+---------+
|customer_id|            name|     city|      state|country|registration_date|is_active|year|month|order_id|order_date|      total_amount|   status|
+-----------+----------------+---------+-----------+-------+-----------------+---------+----+-----+--------+----------+------------------+---------+
|    1000000|Customer_1000000|     Pune|Maharashtra|  India|       2023-07-26|    false|2023|    7| 1502307|2024-05-31| 519.3955458300142|Cancelled|
|    1000011|Customer_1000011|Ahmedabad| Tamil Nadu|  India|       2023-09-22|    false|2023|    9| 2406772|2024-04-15| 471.0864780752916|  Pending|
|    1000012|Customer_1000012|   Mumbai| Tamil Nadu|  India|       2023-07-16|     true|2023|    7| 1479334|2024-07-31| 535.6041079359304|  Shipped|
|    1000012|Customer_1000012|   Mumbai| Tamil Nadu|  India|       2023-07-16|     true|2023|    7| 304869

In [43]:
order_count=customer_orders.groupBy('customer_id').count().orderBy(col('count').desc())
order_count.show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|    1184144|   10|
|     701419|   10|
|    2547389|    9|
|    1748951|    9|
|    2595335|    9|
|    2998605|    9|
|    1204001|    8|
|    1313776|    8|
|    2432381|    8|
|    1204347|    8|
|    2127581|    8|
|    2039671|    8|
|    2565528|    8|
|    1889138|    8|
|    2630731|    8|
|     151032|    8|
|    3234636|    8|
|    1631686|    8|
|    3592692|    8|
|    1530300|    8|
+-----------+-----+
only showing top 20 rows



In [41]:
total_amount=customer_orders.groupBy('customer_id').agg(sum('total_amount').alias('total_amount')).orderBy(col('total_amount').desc())
total_amount.show()

+-----------+------------------+
|customer_id|      total_amount|
+-----------+------------------+
|    2630077| 5936.777119846072|
|    2547389| 5748.199108926318|
|    4532014|5680.0324455917835|
|    2969298| 5546.110744113697|
|    3454033| 5513.938779768613|
|    3281258| 5462.493739728592|
|    4755301|5391.1316191556325|
|    1376165| 5318.097906316698|
|     383604| 5297.194795094891|
|    4349698| 5263.905562703445|
|    3442436| 5190.268361751552|
|    3592692| 5187.701256012082|
|     701419| 5184.141796216243|
|     332670| 5141.417074537108|
|    2592848| 5129.028853499511|
|    2061739| 5076.605204113468|
|    1889138| 5071.704142003546|
|    3210525|5000.0603880647695|
|    3000775| 4999.469241370573|
|    4963794|4999.2740518844985|
+-----------+------------------+
only showing top 20 rows



In [30]:
customer_orders.groupBy('customer_id')\
.agg(avg(col('total_amount')).alias('average_order_amount'))\
     .orderBy(col('average_order_amount').desc()).show()

+-----------+--------------------+
|customer_id|average_order_amount|
+-----------+--------------------+
|     603995|   999.9997827289071|
|    2080633|    999.999683623626|
|    1310263|   999.9994762862821|
|    4614161|    999.999246290303|
|    5076666|   999.9991228320214|
|    4085441|   999.9991055328865|
|    2399523|   999.9989568505075|
|     931809|   999.9988341407084|
|    1026829|   999.9988046710603|
|    3228109|    999.998444936647|
|     421665|   999.9981865214955|
|    4766013|   999.9978617953803|
|    3995436|   999.9973907470671|
|    1924378|   999.9968041189952|
|     705903|   999.9962631927656|
|     672460|   999.9962056271181|
|    3472796|   999.9949015537173|
|    1052542|   999.9945306303302|
|       2578|   999.9931659483154|
|     863479|   999.9930943285091|
+-----------+--------------------+
only showing top 20 rows



In [31]:
customer_orders.groupBy('status').count().show()

+---------+-------+
|   status|  count|
+---------+-------+
|Cancelled|1321657|
|Delivered|1321270|
|  Shipped|1321656|
|  Pending|1322355|
+---------+-------+



In [32]:
customer_orders_with_month=customer_orders.withColumn('Order Month',month('order_date'))\
.groupBy('Order Month').count()
customer_orders_with_month.orderBy(col('Order Month').desc()).show()

+-----------+------+
|Order Month| count|
+-----------+------+
|         12|435509|
|         11|433658|
|         10|449275|
|          9|433563|
|          8|448275|
|          7|449619|
|          6|435084|
|          5|450206|
|          4|434472|
|          3|448743|
|          2|419981|
|          1|448553|
+-----------+------+



In [37]:
from pyspark.sql.window import Window
window_spec=Window.partitionBy('state').orderBy(col('registration_date').desc())
df=df.withColumn('rank',rank().over(window_spec))\
.withColumn('dense rank',dense_rank().over(window_spec))\
.withColumn('row number',row_number().over(window_spec))

In [38]:
df.show()

+-----------+----------------+---------+-----+-------+-----------------+---------+----+-----+----+----------+----------+
|customer_id|            name|     city|state|country|registration_date|is_active|year|month|rank|dense rank|row number|
+-----------+----------------+---------+-----+-------+-----------------+---------+----+-----+----+----------+----------+
|    4145548|Customer_4145548|Bangalore|Delhi|  India|       2023-12-31|     true|2023|   12|   1|         1|         1|
|    4150800|Customer_4150800|     Pune|Delhi|  India|       2023-12-31|     true|2023|   12|   1|         1|         2|
|    4153901|Customer_4153901|  Chennai|Delhi|  India|       2023-12-31|    false|2023|   12|   1|         1|         3|
|    4154641|Customer_4154641|   Mumbai|Delhi|  India|       2023-12-31|     true|2023|   12|   1|         1|         4|
|    4155632|Customer_4155632|   Mumbai|Delhi|  India|       2023-12-31|     true|2023|   12|   1|         1|         5|
|    4157054|Customer_4157054|  

In [39]:
df.tail(10)

[Row(customer_id='5265648', name='Customer_5265648', city='Chennai', state='West Bengal', country='India', registration_date=datetime.date(2023, 1, 1), is_active=True, year=2023, month=1, rank=753355, dense rank=365, row number=755435),
 Row(customer_id='5268272', name='Customer_5268272', city='Chennai', state='West Bengal', country='India', registration_date=datetime.date(2023, 1, 1), is_active=False, year=2023, month=1, rank=753355, dense rank=365, row number=755436),
 Row(customer_id='5268280', name='Customer_5268280', city='Ahmedabad', state='West Bengal', country='India', registration_date=datetime.date(2023, 1, 1), is_active=False, year=2023, month=1, rank=753355, dense rank=365, row number=755437),
 Row(customer_id='5268317', name='Customer_5268317', city='Mumbai', state='West Bengal', country='India', registration_date=datetime.date(2023, 1, 1), is_active=False, year=2023, month=1, rank=753355, dense rank=365, row number=755438),
 Row(customer_id='5270919', name='Customer_52709

In [40]:
customer_orders.show()

+-----------+----------------+---------+-----------+-------+-----------------+---------+----+-----+--------+----------+------------------+---------+
|customer_id|            name|     city|      state|country|registration_date|is_active|year|month|order_id|order_date|      total_amount|   status|
+-----------+----------------+---------+-----------+-------+-----------------+---------+----+-----+--------+----------+------------------+---------+
|    1000000|Customer_1000000|     Pune|Maharashtra|  India|       2023-07-26|    false|2023|    7| 1502307|2024-05-31| 519.3955458300142|Cancelled|
|    1000011|Customer_1000011|Ahmedabad| Tamil Nadu|  India|       2023-09-22|    false|2023|    9| 2406772|2024-04-15| 471.0864780752916|  Pending|
|    1000012|Customer_1000012|   Mumbai| Tamil Nadu|  India|       2023-07-16|     true|2023|    7| 3048691|2024-03-14| 561.5900058615415|  Shipped|
|    1000012|Customer_1000012|   Mumbai| Tamil Nadu|  India|       2023-07-16|     true|2023|    7| 147933

In [42]:
window_spec=Window.orderBy(col('total_amount').desc())
ranked_customers=total_amount.withColumn('rank',rank().over(window_spec))\
.withColumn('dense rank',dense_rank().over(window_spec))
ranked_customers.show()

25/11/08 20:41:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/08 20:41:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/08 20:41:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/08 20:41:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/08 20:41:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/08 20:41:49 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/08 2

+-----------+------------------+----+----------+
|customer_id|      total_amount|rank|dense rank|
+-----------+------------------+----+----------+
|    2630077| 5936.777119846072|   1|         1|
|    2547389| 5748.199108926318|   2|         2|
|    4532014|5680.0324455917835|   3|         3|
|    2969298| 5546.110744113697|   4|         4|
|    3454033| 5513.938779768613|   5|         5|
|    3281258| 5462.493739728592|   6|         6|
|    4755301|5391.1316191556325|   7|         7|
|    1376165| 5318.097906316698|   8|         8|
|     383604| 5297.194795094891|   9|         9|
|    4349698| 5263.905562703446|  10|        10|
|    3442436| 5190.268361751552|  11|        11|
|    3592692| 5187.701256012082|  12|        12|
|     701419| 5184.141796216243|  13|        13|
|     332670| 5141.417074537109|  14|        14|
|    2592848|  5129.02885349951|  15|        15|
|    2061739| 5076.605204113468|  16|        16|
|    1889138| 5071.704142003546|  17|        17|
|    3210525|5000.06

In [44]:
total_amount.show()
order_count.show()



+-----------+------------------+
|customer_id|      total_amount|
+-----------+------------------+
|    2630077| 5936.777119846072|
|    2547389| 5748.199108926318|
|    4532014| 5680.032445591784|
|    2969298| 5546.110744113697|
|    3454033| 5513.938779768613|
|    3281258| 5462.493739728592|
|    4755301|5391.1316191556325|
|    1376165| 5318.097906316698|
|     383604| 5297.194795094891|
|    4349698| 5263.905562703445|
|    3442436| 5190.268361751553|
|    3592692| 5187.701256012082|
|     701419| 5184.141796216244|
|     332670| 5141.417074537108|
|    2592848| 5129.028853499511|
|    2061739| 5076.605204113468|
|    1889138| 5071.704142003546|
|    3210525|5000.0603880647695|
|    3000775| 4999.469241370573|
|    4963794|4999.2740518844985|
+-----------+------------------+
only showing top 20 rows



+-----------+-----+
|customer_id|count|
+-----------+-----+
|    1184144|   10|
|     701419|   10|
|    1748951|    9|
|    2547389|    9|
|    2998605|    9|
|    2595335|    9|
|    1313776|    8|
|    1204347|    8|
|    2039671|    8|
|    1889138|    8|
|     151032|    8|
|    1631686|    8|
|    1530300|    8|
|    1204001|    8|
|     276868|    8|
|    1855567|    8|
|    2952719|    8|
|    1430454|    8|
|    2969298|    8|
|    3210525|    8|
+-----------+-----+
only showing top 20 rows



In [45]:
count_amount=total_amount.join(order_count,'customer_id','inner').orderBy(col('count').desc(),col('total_amount').asc())
count_amount.show()

+-----------+------------------+-----+
|customer_id|      total_amount|count|
+-----------+------------------+-----+
|    1184144| 4420.915631367595|   10|
|     701419| 5184.141796216244|   10|
|    1748951| 4539.175005796893|    9|
|    2595335| 4688.798673753968|    9|
|    2998605| 4988.645805824339|    9|
|    2547389| 5748.199108926318|    9|
|    4493444|2468.2873803978187|    8|
|    3818271| 3000.679484267761|    8|
|     710026| 3001.645507223075|    8|
|    3420580|3017.9987009852607|    8|
|    4499769|3031.7616656169757|    8|
|    1430454|3031.9518853648506|    8|
|     151032| 3226.736327577244|    8|
|    1631686| 3302.902331631487|    8|
|    2890776|3363.9487454785517|    8|
|     754315|3374.4159826379378|    8|
|    1530300| 3474.593195479197|    8|
|    4743307| 3498.293064488408|    8|
|    1204001| 3521.628060978524|    8|
|    3234636| 3559.960370457673|    8|
+-----------+------------------+-----+
only showing top 20 rows

